## Laboratorium 5 - algorytm Najbliższej Średniej (NM)


### Opis
Celem laboratorium jest implementacja klasyfikatora najbliższej średniej NM (*Nearest Mean*).


### Zadanie 1
* Wczytaj dane.
* Wszystkie poniższe zadania wykonaj dla wszystkich dostępnych klas i wszystkich cech.


In [12]:
import numpy as np
with open('./dataset/dataset.npz', 'rb') as f:
    data = np.load(f)
    train, test = data['train'], data['test']

y_train = train[:,0]
X_train = train[:,2:]
y_test = test[:,0]
X_test = test[:,2:]

print(X_train.shape, y_train.shape)

(2244, 14) (2244,)


### Zadanie 2
Zaimplementuj klasyfikator najbliższej średniej (NM) z zastosowaniem odległości Euklidesa i wykonaj klasyfikację. Wyświetl wynik klasyfikacji (accuracy).

In [13]:
from sklearn.metrics import accuracy_score

def nearestMean_fit(X, y):
    centroids = {}
    y_classes = np.unique(y)
    for y_class in y_classes:
        centroids[y_class] = X[y == y_class].mean(axis=0)
    for i, centroid in enumerate(centroids):
        if i % 10 == 0:
            print(f"Klasa {i}: {centroids[centroid]}")
    return centroids, y_classes

def nearestMean_predict(X, centroids, y):
    y_classes = np.unique(y)
    predictions = []
    for x in X:
        distances = {}
        for y_class in y_classes: 
            distances[y_class] = np.linalg.norm(x - centroids[y_class])
        # print(distances)
        pred = min(distances, key=distances.get)
        predictions.append(pred)
    return np.array(predictions)

centroids, y_classes = nearestMean_fit(X_train, y_train)
predictions = nearestMean_predict(X_test, centroids, y_classes)

accuracy = accuracy_score(y_test, predictions)
print(f"\nAccuracy: {accuracy:.2f}")

Klasa 0: [8.04829893e-01 1.70366987e+00 4.25848188e-01 9.73543273e-01
 9.93903017e-01 7.73864549e-01 5.79951347e-03 3.87482009e-03
 2.03802164e-02 7.15419902e-02 6.15755177e-03 1.86811787e-03
 1.02716254e-04 5.83331644e-01]
Klasa 10: [4.28650065e-01 1.07883572e+00 6.65407177e-01 5.32168458e-01
 6.33272125e-01 1.51801982e-01 1.26683428e-01 2.93407580e+00
 2.67988196e-02 8.99281742e-02 8.17501461e-03 2.62608981e-03
 1.35328803e-04 8.09846330e-01]
Klasa 20: [7.80214459e-01 1.56916889e+00 3.88187822e-01 9.88153389e-01
 1.01569482e+00 8.08649353e-01 5.88184477e-03 1.45377038e-02
 3.28249659e-02 9.25700882e-02 8.85827555e-03 2.48925335e-03
 2.88838902e-04 1.11486375e+00]

Accuracy: 0.46


### Zadanie 3
Zaimplementuj funkcję, która zwraca macierz kowariancji (*uwaga: biblioteka `numpy` posiada gotową implementację `cov` z którą powinieneś porównać swój wynik*).

\begin{equation*}
C = \frac{1}{n - 1} (X - \bar X)(X - \bar X)^T
\end{equation*}

gdzie:
* $X$ to macierz danych,
* $\bar X$ to wektor średnich wartości cech. 



In [14]:
def cov_matrix(X):
    X_mean = np.mean(X, axis=0)
    X_centered = X - X_mean
    
    n = X.shape[0]
    C = (1 / (n - 1)) * np.dot(X_centered.T, X_centered)
    
    return C

own_cov = cov_matrix(X_train)
np_cov = np.cov(X_train, rowvar=False)
print(own_cov[0] == np_cov[0])

[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True]


### Zadanie 4
Zaimplementuj klasyfikator najbliższej średniej (NM) z zastosowaniem odległości Mahalanobisa i wykonaj klasyfikację. Wyświetl wynik klasyfikacji (accuracy).

\begin{equation*}
D_j = \sqrt{ (x - \mu_j)^T S_j^{-1}(x - \mu_j) },
\end{equation*}

gdzie:
* $D_j$ to odległość klasyfikowanej próbki od klasy $j$, 
* $\mu_j$ to wektor średnich wartości cech dla klasy $j$, 
* $S_j^{-1}$ to macierz odwrotna do macierzy kowariancji klasy $j$, 
* a $x$ to klasyfikowana próbka.

> Podpowiedź: Do obliczenia macierzy odwrotnej możesz użyć funkcji `linalg.inv` z biblioteki `numpy`.

> UWAGA: W niniejszym zadaniu możesz zastosować dowolną strukturę kodu (nie musisz trzymać się struktury z poprzedniego zadania), jednak algorytm NM należy zaimplementować samodzielnie – bez użycia gotowych rozwiązań (np. z biblioteki `scikit-learn`).

<span style="text-decoration:underline">Referencje</span>

1. Mahalanobis, P C, _On test and measures of group divergence : theoretical formulae_, Journal and Proceedings of Asiatic Society of Bengal (New Series) Vol. 26, pp. 541-588. 1930. (URL: http://library.isical.ac.in:8080/xmlui/bitstream/handle/10263/1639/029.pdf)
2. McLachlan, Goeffrey J. _Mahalanobis distance_, Resonance, pp. 20-26. 1999. (URL: https://www.ias.ac.in/article/fulltext/reso/004/06/0020-0026)

In [17]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

def mahalanobis_distance(x, centroid, inv_cov):
    diff = x - centroid
    distance = np.sqrt(diff.T @ inv_cov @ diff)
    return distance

def nearestMean_fit(X, y):
    centroids = {}
    inv_covs = {} 
    y_classes = np.unique(y)

    for y_class in y_classes:
        X_class = X[y == y_class]
        centroids[y_class] = X_class.mean(axis=0)
        
        cov = np.cov(X_class, rowvar=False)
        inv_covs[y_class] = np.linalg.inv(cov)
    return centroids, inv_covs, y_classes

def nearestMean_predict(X, centroids, inv_covs, y):
    y_classes = np.unique(y)
    predictions = []
    for x in X:
        distances = {}
        for y_class in y_classes:
            distances[y_class] = mahalanobis_distance(x, centroids[y_class], inv_covs[y_class])
        pred = min(distances, key=distances.get)
        predictions.append(pred)
    
    return np.array(predictions)

centroids, inv_covs, y_classes = nearestMean_fit(X_train, y_train)

predictions = nearestMean_predict(X_test, centroids, inv_covs, y_train)
print(f"Skuteczność: {accuracy_score(y_test, predictions)*100:.2f}%")

Skuteczność: 56.35%
